# Download das nuscenes mini

In [ ]:
import os
import subprocess

def download(url, path):

    # comando para fazer o download
    cmd = ['wget', '-q', url, '-O', '%s' %(path)]

    # executa o comando sem exibir a saída na tela
    processo = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # aguarda o término do download
    stdout, stderr = processo.communicate()

    # Verifica se o processo foi concluído com sucesso (código de retorno zero)
    if processo.returncode == 0:
        print(f'Processo finalizado.')
    else:
        print(f'Houve um erro: {stderr.decode("utf-8")}')

# especifica o local onde ficarao os arquivos
pathFiles = 'dados/'

# cria uma pasta onde ficarao os arquivos
if not os.path.isdir(pathFiles):
    os.mkdir(pathFiles)

url = 'https://www.nuscenes.org/data/v1.0-mini.tgz'
pathDataset1 = pathFiles + '/nuscenes_v1.0-mini.tgz'
download(url, pathDataset1)

Processo finalizado.


Descompacta a base de dados.

In [ ]:
import tarfile

def descompacta(path, pathFolder):

    try:
        # Descompacta ao arquivo
        with tarfile.open(pathDataset1, 'r:gz') as tar:
          tar.extractall(path=pathFolder)

        print("Arquivo descompactado com sucesso!")
    except:
        print("Houve um erro ao tentar descompactar o arquivo")

pathFiles = pathFiles + "/nuscenes"

# Cria o diretório de extração se não existir
os.makedirs(pathFiles, exist_ok=True)

descompacta(pathDataset1, pathFiles)

Arquivo descompactado com sucesso!


Instala a nuscenes

In [ ]:
!pip install nuscenes-devkit &> /dev/null  # Install nuScenes.

# Converte uma bounding box 3D para 2D

A função abaixo recebe as informações da bounding box 3D de um determinado objeto e as informações de calibração da câmera. Com base nisso, ela faz a conversão para 2D. Esse código foi baseado no notebook disponível nesse link:
- <https://github.com/asvath/mobile_robotics/blob/master/nuscenes%20extract%20and%20write%20out%202d%20annotation%20boxes-revised%20to%20truncate%20bb.ipynb>

In [ ]:
import numpy as np
import pandas as pd
import cv2
from nuscenes.nuscenes import NuScenes

from pyquaternion import Quaternion

from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import view_points, BoxVisibility, box_in_image

import matplotlib.pyplot as plt

def threeD_2_twoD(boxsy,intrinsic): #input is a single annotation box
    '''
    given annotation boxes and intrinsic camera matrix
    outputs the 2d bounding box coordinates as a list (all annotations for a particular sample image)
    '''
    corners = boxsy.corners()
    x = corners[0,:]
    y = corners[1,:]
    z = corners[2,:]
    x_y_z = np.array((x,y,z))
    orthographic = np.dot(intrinsic,x_y_z)
    perspective_x = orthographic[0]/orthographic[2]
    perspective_y = orthographic[1]/orthographic[2]
    perspective_z = orthographic[2]/orthographic[2]

    min_x = int(np.min(perspective_x))
    max_x = int(np.max(perspective_x))
    min_y = int(np.min(perspective_y))
    max_y = int(np.max(perspective_y))

    return min_x,max_x,min_y,max_y

def getBoundBox(nusc, sample, camera, annotation_metadata):
    """
    Obtém a bounding box 3D e, depois, usando as informações da câmera,
    chama a função que converte para 2D
    """

    camera_token = sample['data'][camera]

    # Carregue os dados da câmera
    cam_data = nusc.get('sample_data', camera_token)

    cam_intrinsics = np.array(nusc.get('calibrated_sensor', cam_data['calibrated_sensor_token'])['camera_intrinsic'])
    cam_height = cam_data['height']
    cam_width = cam_data['width']

    # obtenha a bounding box 3D do pedestre
    # a variavel ann foi filtrada a partir do token do pedestre. Portanto, o parametro translation contem
    # as coordenadas do pedestre em questao
    translation = annotation_metadata['translation']
    size = annotation_metadata['size']
    rotation = Quaternion(annotation_metadata['rotation'])
    bbox = Box(translation, size, rotation)

    # Obtenha a pose da câmera
    ego_pose = nusc.get('ego_pose', cam_data['ego_pose_token'])
    cam_pose = nusc.get('calibrated_sensor', cam_data['calibrated_sensor_token'])

    # Transforme a bounding box do pedestre para o sistema de coordenadas da câmera
    bbox.translate(-np.array(ego_pose['translation']))
    bbox.rotate(Quaternion(ego_pose['rotation']).inverse)
    bbox.translate(-np.array(cam_pose['translation']))
    bbox.rotate(Quaternion(cam_pose['rotation']).inverse)

    # Verificar se a bounding box está dentro do campo de visão da câmera
    if not box_in_image(bbox, cam_intrinsics, imsize=[cam_width, cam_height], vis_level=BoxVisibility.ANY):
        return None, None, None, None

    # converts box into image plane
    min_x, max_x, min_y, max_y = threeD_2_twoD(bbox,cam_intrinsics)

    return min_x, max_x, min_y, max_y

A função abaixo aplica a conversão das bounding box para todas as anotações da base de dados. Além disso, guarda todas as informações em um dataframe.

In [ ]:
def get_scenes(nusc):

    columns = ['scene_token', 'category_name', 'instance_token', 'translation', 'size', 'rotation']
    annotations_df = pd.DataFrame(columns=columns)

    dataList = []
    for scene in nusc.scene:
        scene_token = scene['token']
        print(f"Processing scene: {scene_token}")  # Adicione esta linha para depuração
        sample_token = scene['first_sample_token']

        while sample_token:
            sample = nusc.get('sample', sample_token)

            scene_token = sample['scene_token']
            timestamp = sample['timestamp']

            # coletar todos os caminhos das imagens da câmera
            for sensor_channel in ['CAM_FRONT', 'CAM_BACK', 'CAM_FRONT_LEFT', 'CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT']:
                sample_data_token_camera = sample['data'][sensor_channel]
                sample_data_camera = nusc.get('sample_data', sample_data_token_camera)
                image_path = sample_data_camera['filename']


                for annotation_token in sample['anns']:
                    annotation_metadata = nusc.get('sample_annotation', annotation_token)
                    instance_metadata = nusc.get('instance', annotation_metadata['instance_token'])
                    category_metadata = nusc.get('category', instance_metadata['category_token'])

                    # visibility_token = annotation_metadata['visibility_token']

                    # obter descrição da visibilidade
                    # visibility = nusc.get('visibility', visibility_token)['description']

                    min_x, max_x, min_y, max_y = getBoundBox(nusc, sample, sensor_channel, annotation_metadata)

                    data = {
                        'sample_token_camera': sample_data_token_camera,
                        'sample_token': sample_token,
                        'scene_token': scene_token,  # Certifique-se de que `scene_token` está sendo atribuído aqui
                        'instance_token': annotation_metadata['instance_token'],
                        'category_name': category_metadata['name'],
                        'image_path': image_path,
                        'camera': sensor_channel,
                        'translation': annotation_metadata['translation'],
                        'size': annotation_metadata['size'],
                        'rotation': annotation_metadata['rotation'],
                        'timestamp': timestamp,
                        # 'box_visibility': visibility_token,
                        'bbox_min_x': min_x,
                        'bbox_max_x': max_x,
                        'bbox_min_y': min_y,
                        'bbox_max_y': max_y,
                    }

                    dataList.append(data)

            sample_token = sample['next']

    # converta para um dataframe
    annotations_df = pd.DataFrame(dataList)

    # Ordenar pelo campo 'timestamp'
    annotations_df = annotations_df.sort_values(by='timestamp')

    annotations_df.to_csv('annotations_with_scenes.csv', index=False)

    return annotations_df

Converte toda a base de dados e salva como .csv

In [ ]:
if __name__ == "__main__":

    pathDataset = "/content/dados/nuscenes/"

    path = "dados/annotations_bbox2D.csv"

    nusc = NuScenes(version='v1.0-mini', dataroot=pathDataset, verbose=True)

    annotations_df = get_scenes(nusc)

    annotations_df.to_csv(path, index=False)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 1.164 seconds.
Reverse indexing ...
Done reverse indexing in 0.2 seconds.
Processing scene: cc8c0bf57f984915a77078b10eb33198
Processing scene: fcbccedd61424f1b85dcbf8f897f9754
Processing scene: 6f83169d067343658251f72e1dd17dbc
Processing scene: bebf5f5b2a674631ab5c88fd1aa9e87a
Processing scene: 2fc3753772e241f2ab2cd16a784cc680
Processing scene: c5224b9b454b4ded9b5d2d2634bbda8a
Processing scene: 325cef682f064c55a255f2625c533b75
Processing scene: d25718445d89453381c659b9c8734939
Processing scene: de7d80a1f5fb4c3e82ce8a4f213b450a
Processing scene: e233467e827140efa4b42d2b4c435855


# Filtra a base de dados

A função abaixo filtra apenas algumas categorias de interesse.


In [ ]:
def filtraCategorias(annotations_df):
    # Definir as categorias de interesse - apenas pedestres
    categories_of_interest = [
        'human.pedestrian.adult',
        'human.pedestrian.child',
        'human.pedestrian.construction_worker',
        'human.pedestrian.personal_mobility',
        'human.pedestrian.police_officer'
    ]

    # Filtrar as anotações que correspondem a essas categorias
    filtered_annotations_df = annotations_df[annotations_df['category_name'].isin(categories_of_interest)]

    return filtered_annotations_df


A função abaixo remove as bounding box nulas. Depois, aplica as duas funções.

In [ ]:
def remove_bbox_nula(df):
    """
    Remove as linhas onde a bounding box é nula
    """

    df = df.dropna(subset=['bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y'])

    return df


Aplica a função que seleciona algumas categorias e a função que remove as bounding box nulas.

In [ ]:
if __name__ == "__main__":

  path_filtered = "dados/annotations_bbox2D_filtered.csv"

  # aplica a função que filtra as categorias desejadas
  annotations_df_filtered = filtraCategorias(annotations_df)

  # aplica a função que remove as bouding box nulas
  annotations_df_filtered = remove_bbox_nula(annotations_df_filtered)

  # salva o dataframe
  annotations_df_filtered.to_csv(path_filtered, index=False)

#Processamento de Dados e Função de Calculo de IoU

In [ ]:
import pandas as pd

# Carrega o DataFrame gerado com as anotações das bounding boxes
annotations_df = pd.read_csv('/content/dados/annotations_bbox2D_filtered.csv')

# Ordenar pelo 'instance_token' e 'timestamp' para garantir que os dados estejam na ordem correta
annotations_df = annotations_df.sort_values(by=['instance_token', 'timestamp'])

# Shift para obter o bbox do timestamp anterior e criar colunas de diferença
annotations_df['prev_bbox_min_x'] = annotations_df.groupby('instance_token')['bbox_min_x'].shift(1)
annotations_df['prev_bbox_max_x'] = annotations_df.groupby('instance_token')['bbox_max_x'].shift(1)
annotations_df['prev_bbox_min_y'] = annotations_df.groupby('instance_token')['bbox_min_y'].shift(1)
annotations_df['prev_bbox_max_y'] = annotations_df.groupby('instance_token')['bbox_max_y'].shift(1)

# Remove as linhas onde não temos dados prévios
annotations_df = annotations_df.dropna(subset=['prev_bbox_min_x', 'prev_bbox_max_x', 'prev_bbox_min_y', 'prev_bbox_max_y'])

# Definir as features (inputs) e a variável alvo (outputs)
X = annotations_df[['prev_bbox_min_x', 'prev_bbox_max_x', 'prev_bbox_min_y', 'prev_bbox_max_y',
                    'bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y']]
y = annotations_df[['bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y']]


In [ ]:
def calcula_iou(bbox_real, bbox_predito):
    """
    Calcula o IoU (Intersection over Union) entre duas bounding boxes.
    """
    xA = max(bbox_real[0], bbox_predito[0])
    yA = max(bbox_real[2], bbox_predito[2])
    xB = min(bbox_real[1], bbox_predito[1])
    yB = min(bbox_real[3], bbox_predito[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxA_area = (bbox_real[1] - bbox_real[0] + 1) * (bbox_real[3] - bbox_real[2] + 1)
    boxB_area = (bbox_predito[1] - bbox_predito[0] + 1) * (bbox_predito[3] - bbox_predito[2] + 1)

    iou = interArea / float(boxA_area + boxB_area - interArea)
    return iou

def calcula_iou_medio(y_true, y_pred):
    """
    Calcula o IoU médio entre todos os pares de bounding boxes.
    """
    iou_sum = 0
    count = len(y_true)

    for i in range(count):
        iou_sum += calcula_iou(y_true[i], y_pred[i])

    return iou_sum / count


#Criação e Teste dos modelos

##Modelo para prever a bounding box do próximo frame

In [ ]:
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import tensorflow as tf
import random
import os

# Função para definir a seed global para reprodução dos resultados
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

    # Configurações adicionais para reprodução consistente no TensorFlow
    os.environ['PYTHONHASHSEED'] = str(seed_value)  # Hash seed
    os.environ['TF_DETERMINISTIC_OPS'] = '1'  # Força operações determinísticas no TensorFlow

    # Configurações adicionais para TensorFlow
    # Definir configuração para limitar o uso de threads paralelos
    tf.config.threading.set_intra_op_parallelism_threads(1)
    tf.config.threading.set_inter_op_parallelism_threads(1)

# Chamar a função para definir a seed
set_seed()

# Definir os hiperparâmetros a serem testados
hidden_units = [
    (128, 64, 32, 16, 8), (64, 32, 16, 8), (32, 16, 8), (16, 8), (64, 64), (64, 32), (32, 32)]  # Diferentes tamanhos de camadas ocultas
activations = ['linear', 'relu', 'leaky_relu', 'selu', 'swish', 'mish']  # Funções de ativação
learning_rates = [0.0001, 0.001, 0.01]  # Diferentes taxas de aprendizado
epochs_list = [10, 20, 30, 40, 50]  # Diferentes números de épocas a serem testados

# Dividir em treino e teste mantendo a ordem temporal
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]



# Função para criar o modelo com hiperparâmetros variáveis
def create_model(hidden_units, activation, learning_rate):
    # Definir os inputs da rede (bbox no timestamp anterior e atual)
    input_prev = Input(shape=(4,), name='input_prev_bbox')
    input_current = Input(shape=(4,), name='input_current_bbox')

    # Definir o ramo siamesa compartilhado
    def siamese_branch(input_layer):
        x = input_layer
        for units in hidden_units:
            x = Dense(units, activation=activation)(x)
        return x

    # Aplicar o ramo siamesa para as duas entradas
    output_prev = siamese_branch(input_prev)
    output_current = siamese_branch(input_current)

    # Concatenar as saídas dos dois ramos
    merged = Concatenate()([output_prev, output_current])

    # Camada de saída para prever a bbox futura
    output = Dense(4, activation='linear')(merged)

    # Criar o modelo
    model = Model(inputs=[input_prev, input_current], outputs=output)

    # Compilar o modelo
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    return model

# Armazenar os melhores resultados
best_model = None
best_iou = 0
best_hyperparams = {}

# Loop pelos hiperparâmetros
for hidden in hidden_units:
    for activation in activations:
        for lr in learning_rates:
            for epochs in epochs_list:
                print(f"Testando modelo com {hidden} camadas ocultas, ativação {activation}, lr {lr}, epochs {epochs}")

                # Criar o modelo com os hiperparâmetros atuais
                model = create_model(hidden, activation, lr)

                # Treinar o modelo
                model.fit([X_train[['prev_bbox_min_x', 'prev_bbox_max_x', 'prev_bbox_min_y', 'prev_bbox_max_y']],
                           X_train[['bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y']]],
                          y_train, epochs=epochs, batch_size=32, verbose=0)

                # Fazer predições
                predictions = model.predict([X_test[['prev_bbox_min_x', 'prev_bbox_max_x', 'prev_bbox_min_y', 'prev_bbox_max_y']],
                                             X_test[['bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y']]])

                # Calcular o IoU médio
                iou_medio = calcula_iou_medio(y_test.values, predictions)
                print(f"IoU médio: {iou_medio}")

                # Verificar se este é o melhor modelo
                if iou_medio > best_iou:
                    best_iou = iou_medio
                    best_model = model
                    best_hyperparams = {
                        'hidden_units': hidden,
                        'activation': activation,
                        'learning_rate': lr,
                        'epochs': epochs
                    }

# Exibir os melhores hiperparâmetros
print(f"Melhor combinação de hiperparâmetros: {best_hyperparams}")
print(f"Melhor IoU médio: {best_iou}")

Testando modelo com (128, 64, 32, 16, 8) camadas ocultas, ativação linear, lr 0.0001, epochs 10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
IoU médio: 0.7568891821607336
Testando modelo com (128, 64, 32, 16, 8) camadas ocultas, ativação linear, lr 0.0001, epochs 20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
IoU médio: 0.7933421115150642
Testando modelo com (128, 64, 32, 16, 8) camadas ocultas, ativação linear, lr 0.0001, epochs 30
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
IoU médio: 0.8285111300752686
Testando modelo com (128, 64, 32, 16, 8) camadas ocultas, ativação linear, lr 0.0001, epochs 40
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
IoU médio: 0.8699910158412525
Testando modelo com (128, 64, 32, 16, 8) camadas ocultas, ativação linear, lr 0.0001, epochs 50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
IoU médio: 0.8640613593926028
Testando modelo com (128, 64, 32, 16, 8) camadas ocultas, ativação linear, lr 0.001, epochs 10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
IoU médio: 0.7971168069003554
Testando mo

##Modelos para prever a bounding box:
- Dos próximos 5 frames, com um histórico de 5 frames como input.</br>
- Dos próximos 5 frames, com um histórico de 10 frames como input.</br>
- Dos próximos 10 frames, com um histórico de 5 frames como input.</br>
- Dos próximos 10 frames, com um histórico de 10 frames como input.</br>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.optimizers import Adam

# Funções reutilizadas de seu código anterior:
def calcula_iou(bbox_real, bbox_predito):
    # Função para calcular IoU
    xA = max(bbox_real[0], bbox_predito[0])
    yA = max(bbox_real[2], bbox_predito[2])
    xB = min(bbox_real[1], bbox_predito[1])
    yB = min(bbox_real[3], bbox_predito[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxA_area = (bbox_real[1] - bbox_real[0] + 1) * (bbox_real[3] - bbox_real[2] + 1)
    boxB_area = (bbox_predito[1] - bbox_predito[0] + 1) * (bbox_predito[3] - bbox_predito[2] + 1)

    iou = interArea / float(boxA_area + boxB_area - interArea)
    return iou

def calcula_iou_medio(y_true, y_pred):
    iou_sum = 0
    count = len(y_true)

    for i in range(count):
        iou_sum += calcula_iou(y_true[i], y_pred[i])

    return iou_sum / count

def create_model(input_shape, output_shape, hidden_units, activation, learning_rate):
    input_prev = Input(shape=input_shape, name='input_prev_bbox')

    # Definir o ramo siamesa compartilhado
    def siamese_branch(input_layer):
        x = input_layer
        for units in hidden_units:
            x = Dense(units, activation=activation)(x)
        return x

    # Aplicar o ramo siamesa para as entradas
    output_prev = siamese_branch(input_prev)

    # Camada de saída para prever a bbox futura
    output = Dense(output_shape[0], activation='linear')(output_prev)

    # Criar o modelo
    model = Model(inputs=[input_prev], outputs=output)

    # Compilar o modelo
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    return model

# Função para formatar os dados de acordo com o número de frames de input/output
def format_data(annotations_df, num_input_frames, num_output_frames):
    X = []
    y = []

    # Garantir que os dados sejam ordenados por scene_token, instance_token e timestamp
    annotations_df = annotations_df.sort_values(by=['scene_token', 'instance_token', 'timestamp'])

    # Criar as sequências de input/output para cada VRU dentro de uma cena
    grouped = annotations_df.groupby(['scene_token', 'instance_token'])

    for _, group in grouped:
        # Processar cada grupo (cada combinação de scene_token e instance_token)
        group = group.reset_index(drop=True)

        # Criar sequências respeitando o número de input e output frames
        for i in range(num_input_frames, len(group) - num_output_frames):
            X.append(group.iloc[i-num_input_frames:i][['bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y']].values.flatten())
            y.append(group.iloc[i:i+num_output_frames][['bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y']].values.flatten())

    return np.array(X), np.array(y)

# Configurações dos quatro cenários
scenarios = [
    {'input_frames': 5, 'output_frames': 5},
    {'input_frames': 5, 'output_frames': 10},
    {'input_frames': 10, 'output_frames': 5},
    {'input_frames': 10, 'output_frames': 10}
]

# Hiperparâmetros para testar
hidden_units = [(128, 64, 32, 16, 8), (64, 32, 16, 8), (32, 16, 8), (16, 8), (64, 64), (64, 32), (32, 32)]
activations = ['linear', 'relu', 'leaky_relu', 'selu', 'swish', 'mish']  # Funções de ativação
learning_rates = [0.0001, 0.001, 0.01]
epochs_list = [10, 20, 30, 40, 50]

# Variáveis para armazenar os melhores resultados por cenário
best_models_per_scenario = {}

# Loop pelos cenários
for scenario in scenarios:
    print(f"Testando cenário: {scenario['input_frames']} input, {scenario['output_frames']} output")

    # Formatando os dados para este cenário
    X, y = format_data(annotations_df, scenario['input_frames'], scenario['output_frames'])

    # Dividir em treino e teste mantendo a ordem temporal
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Variáveis para armazenar o melhor resultado deste cenário
    best_model = None
    best_iou = 0
    best_hyperparams = {}

    # Loop pelos hiperparâmetros
    for hidden in hidden_units:
        for activation in activations:
            for lr in learning_rates:
                for epochs in epochs_list:
                    print(f"Testando modelo com {hidden} camadas ocultas, ativação {activation}, lr {lr}, epochs {epochs}")

                    # Criar o modelo com os hiperparâmetros atuais
                    model = create_model(X_train.shape[1:], y_train.shape[1:], hidden, activation, lr)

                    # Treinar o modelo
                    model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=0)

                    # Fazer predições
                    predictions = model.predict(X_test)

                    # Calcular o IoU médio
                    iou_medio = calcula_iou_medio(y_test, predictions)
                    print(f"IoU médio: {iou_medio}")

                    # Verificar se este é o melhor modelo para o cenário atual
                    if iou_medio > best_iou:
                        best_iou = iou_medio
                        best_model = model
                        best_hyperparams = {
                            'hidden_units': hidden,
                            'activation': activation,
                            'learning_rate': lr,
                            'epochs': epochs
                        }

    # Armazenar o melhor modelo e hiperparâmetros para este cenário
    best_models_per_scenario[scenario['input_frames'], scenario['output_frames']] = {
        'best_model': best_model,
        'best_iou': best_iou,
        'best_hyperparams': best_hyperparams
    }

# Exibir os melhores hiperparâmetros e cenários
for scenario_key, best_model_data in best_models_per_scenario.items():
    print(f"Melhor cenário: Input Frames: {scenario_key[0]}, Output Frames: {scenario_key[1]}")
    print(f"Melhor combinação de hiperparâmetros: {best_model_data['best_hyperparams']}")
    print(f"Melhor IoU médio: {best_model_data['best_iou']}")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.optimizers import Adam

# Funções reutilizadas de seu código anterior:
def calcula_iou(bbox_real, bbox_predito):
    # Função para calcular IoU
    xA = max(bbox_real[0], bbox_predito[0])
    yA = max(bbox_real[2], bbox_predito[2])
    xB = min(bbox_real[1], bbox_predito[1])
    yB = min(bbox_real[3], bbox_predito[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxA_area = (bbox_real[1] - bbox_real[0] + 1) * (bbox_real[3] - bbox_real[2] + 1)
    boxB_area = (bbox_predito[1] - bbox_predito[0] + 1) * (bbox_predito[3] - bbox_predito[2] + 1)

    iou = interArea / float(boxA_area + boxB_area - interArea)
    return iou

def calcula_iou_medio(y_true, y_pred):
    iou_sum = 0
    count = len(y_true)

    for i in range(count):
        iou_sum += calcula_iou(y_true[i], y_pred[i])

    return iou_sum / count

def create_model(input_shape, output_shape, hidden_units, activation, learning_rate):
    input_prev = Input(shape=input_shape, name='input_prev_bbox')

    # Definir o ramo siamesa compartilhado
    def siamese_branch(input_layer):
        x = input_layer
        for units in hidden_units:
            x = Dense(units, activation=activation)(x)
        return x

    # Aplicar o ramo siamesa para as entradas
    output_prev = siamese_branch(input_prev)

    # Camada de saída para prever a bbox futura
    output = Dense(output_shape[0], activation='linear')(output_prev)

    # Criar o modelo
    model = Model(inputs=[input_prev], outputs=output)

    # Compilar o modelo
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    return model

# Função para formatar os dados de acordo com o número de frames de input/output
def format_data(annotations_df, num_input_frames, num_output_frames):
    X = []
    y = []

    # Garantir que os dados sejam ordenados por scene_token, instance_token e timestamp
    annotations_df = annotations_df.sort_values(by=['scene_token', 'instance_token', 'timestamp'])

    # Criar as sequências de input/output para cada VRU dentro de uma cena
    grouped = annotations_df.groupby(['scene_token', 'instance_token'])

    for _, group in grouped:
        # Processar cada grupo (cada combinação de scene_token e instance_token)
        group = group.reset_index(drop=True)

        # Criar sequências respeitando o número de input e output frames
        for i in range(num_input_frames, len(group) - num_output_frames):
            X.append(group.iloc[i-num_input_frames:i][['bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y']].values.flatten())
            y.append(group.iloc[i:i+num_output_frames][['bbox_min_x', 'bbox_max_x', 'bbox_min_y', 'bbox_max_y']].values.flatten())

    return np.array(X), np.array(y)

# Configurações dos quatro cenários
scenarios = [
    {'input_frames': 5, 'output_frames': 1},
    {'input_frames': 10, 'output_frames': 1},
    {'input_frames': 15, 'output_frames': 1}
]

# Hiperparâmetros para testar
hidden_units = [(128, 64, 32, 16, 8), (64, 32, 16, 8), (32, 16, 8), (16, 8), (64, 64), (64, 32), (32, 32)]
activations = ['linear', 'relu', 'leaky_relu', 'selu', 'swish', 'mish']  # Funções de ativação
learning_rates = [0.0001, 0.001, 0.01]
epochs_list = [10, 20, 30, 40, 50]

# Variáveis para armazenar os melhores resultados por cenário
best_models_per_scenario = {}

# Loop pelos cenários
for scenario in scenarios:
    print(f"Testando cenário: {scenario['input_frames']} input, {scenario['output_frames']} output")

    # Formatando os dados para este cenário
    X, y = format_data(annotations_df, scenario['input_frames'], scenario['output_frames'])

    # Dividir em treino e teste mantendo a ordem temporal
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Variáveis para armazenar o melhor resultado deste cenário
    best_model = None
    best_iou = 0
    best_hyperparams = {}

    # Loop pelos hiperparâmetros
    for hidden in hidden_units:
        for activation in activations:
            for lr in learning_rates:
                for epochs in epochs_list:
                    print(f"Testando modelo com {hidden} camadas ocultas, ativação {activation}, lr {lr}, epochs {epochs}")

                    # Criar o modelo com os hiperparâmetros atuais
                    model = create_model(X_train.shape[1:], y_train.shape[1:], hidden, activation, lr)

                    # Treinar o modelo
                    model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=0)

                    # Fazer predições
                    predictions = model.predict(X_test)

                    # Calcular o IoU médio
                    iou_medio = calcula_iou_medio(y_test, predictions)
                    print(f"IoU médio: {iou_medio}")

                    # Verificar se este é o melhor modelo para o cenário atual
                    if iou_medio > best_iou:
                        best_iou = iou_medio
                        best_model = model
                        best_hyperparams = {
                            'hidden_units': hidden,
                            'activation': activation,
                            'learning_rate': lr,
                            'epochs': epochs
                        }

    # Armazenar o melhor modelo e hiperparâmetros para este cenário
    best_models_per_scenario[scenario['input_frames'], scenario['output_frames']] = {
        'best_model': best_model,
        'best_iou': best_iou,
        'best_hyperparams': best_hyperparams
    }

# Exibir os melhores hiperparâmetros e cenários
for scenario_key, best_model_data in best_models_per_scenario.items():
    print(f"Melhor cenário: Input Frames: {scenario_key[0]}, Output Frames: {scenario_key[1]}")
    print(f"Melhor combinação de hiperparâmetros: {best_model_data['best_hyperparams']}")
    print(f"Melhor IoU médio: {best_model_data['best_iou']}")